# UrbanSound8K Data Augmentation

In [1]:
import os
import librosa
import librosa.display
import pandas as pd
import numpy as np
import random
import matplotlib as plt
import matplotlib.pyplot as plt

### Load metadata CSV

In [15]:
# Define general variables

# Set your path to the dataset
us8k_path = os.path.abspath('./UrbanSound8K')
audio_path = os.path.join(us8k_path, 'audio')
augmented_path = os.path.abspath('augmented')

# Metadata
metadata_path = os.path.join(us8k_path, 'metadata/UrbanSound8K.csv')
metadata_augmented_path = os.path.abspath('data/augmented-data.csv')

#### Load metadata

In [3]:
# Load the metadata from the generated CSV
metadata = pd.read_csv(metadata_path)

# Examine dataframe
metadata.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


### Time stretching

#### Time stretching (changing play time)

In [4]:
rates = [0.81, 1.07]
for rate in rates: 
    # Generate new stretched audio file
    for index, row in metadata.iterrows():        
        curr_fold = str(row['fold'])
        curr_file_path = audio_path + '/fold' + curr_fold + '/' + row['slice_file_name']
        
        # Speed sub-dir inside current fold dir
        curr_rate_path = augmented_path + '/fold' + curr_fold + '/speed_' + str(int(rate*100))

        # Create sub-dir if it does not exist
        if not os.path.exists(curr_rate_path):
            os.makedirs(curr_rate_path)
        
        output_path = curr_rate_path + '/' + row['slice_file_name']
        
        y, sr = librosa.load(curr_file_path)  
        y_changed = librosa.effects.time_stretch(y, rate=rate)
        librosa.output.write_wav(output_path, y_changed, sr)


### Pitch shifting

In [5]:
tone_steps = [-1, -2, -2.5, -3.5, 1, 2, 2.5, 3.5]

for tone_step in tone_steps:
    # Generate new pitched audio
    for index, row in metadata.iterrows():        
        curr_fold = str(row['fold'])
        curr_file_path = audio_path + '/fold' + curr_fold + '/' + row['slice_file_name']

        # Pitch Shift sub-dir inside current fold dir
        curr_ps_path = augmented_path + '/fold' + curr_fold + '/pitch_' + str(tone_step)

        # Create sub-dir if it does not exist
        if not os.path.exists(curr_ps_path):
            os.makedirs(curr_ps_path)
        
        output_path = curr_ps_path + '/' + row['slice_file_name']
        
        # Skip when file already exists
        if (os.path.isfile(output_path)):
            print(output_path, " already exists")
            continue
        
        y, sr = librosa.load(curr_file_path)  
        y_changed = librosa.effects.pitch_shift(y, sr, n_steps=tone_step)
        librosa.output.write_wav(output_path, y_changed, sr)
    
    print("Finished tone ", tone_step) 

Finished tone  -1
Finished tone  -2
Finished tone  -2.5
Finished tone  -3.5
Finished tone  1
Finished tone  2
Finished tone  2.5
Finished tone  3.5


### Add noise

In [6]:
import random

def add_noise(data):
    noise = np.random.rand(len(data))
    noise_amp = random.uniform(0.005, 0.008)
    data_noise = data + (noise_amp * noise)
    return data_noise

# Generate new noised audio
for index, row in metadata.iterrows():        
    curr_fold = str(row['fold'])
    curr_file_path = audio_path + '/fold' + curr_fold + '/' + row['slice_file_name']

    # Noised sub-dir inside current fold dir
    curr_noise_path = augmented_path + '/fold' + curr_fold + '/noise'

    # Create sub-dir if it does not exist
    if not os.path.exists(curr_noise_path):
        os.makedirs(curr_noise_path)
        
    output_path = curr_noise_path + '/' + row['slice_file_name']
        
    # Skip when file already exists
    if (os.path.isfile(output_path)):
        print(output_path, " already exists")
        continue
        
    y, sr = librosa.load(curr_file_path)  
    y_changed = add_noise(y)
    librosa.output.write_wav(output_path, y_changed, sr)

### Create metadata for the new files

In [7]:
def get_files_recursive(path):
    # create a list of file and sub directories 
    # names in the given directory 
    file_list = os.listdir(path)
    all_files = list()
    # Iterate over all the entries
    for entry in file_list:
        # Create full path
        full_path = os.path.join(path, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(full_path):
            all_files = all_files + get_files_recursive(full_path)
        else:
            all_files.append(full_path)
                
    return all_files   

In [8]:
# Get every single file within the tree
files = get_files_recursive(augmented_path)

# Define metadata columns
names = []
classes = []
folds = []
augmentations = []

# Iterate and collect name, fold and class
for file in files:
    pieces = file.split("/")
    file = pieces[len(pieces) - 1]
    fold = pieces[len(pieces) - 3] 
    augment = pieces[len(pieces) - 2] 
    fold_num = fold[4:len(fold)]
    class_id = file.split("-")[1]

    # Push records
    names.append(file)
    folds.append(fold_num)
    classes.append(class_id)
    augmentations.append(augment)

# Create a dataframe with the new augmented data
new_meta = pd.DataFrame({'file': names, 'fold': folds, 'class_id': classes, 'augment': augmentations })

# Make sure class_id is int
new_meta['class_id'] = new_meta['class_id'].astype(np.int64)

print(len(new_meta), "new entries")

96052 new entries


In [9]:
# Add class names to the new dataframe using merge
classes = pd.DataFrame({
    'class_id': range(0,10),
    'class': [
        'air_conditioner',
        'car_horn',
        'children_playing',
        'dog_bark',
        'drilling',
        'engine_idling',
        'gun_shot',
        'jackhammer',
        'siren',
        'street_music'
    ]
})

new_meta = pd.merge(new_meta, classes, on='class_id')


In [13]:
new_meta.tail()

,file,fold,class_id,augment,class
96047,159705-6-0-0.wav,7,6,pitch_2.5,gun_shot
96048,135527-6-14-9.wav,7,6,pitch_2.5,gun_shot
96049,159708-6-2-0.wav,7,6,pitch_2.5,gun_shot
96050,159708-6-6-0.wav,7,6,pitch_2.5,gun_shot
96051,148827-6-0-0.wav,7,6,pitch_2.5,gun_shot


### Integrate metadata in a single dataset

In [11]:
# Modify original data to fit the new structure
del metadata['fsID'], metadata['start'], metadata['end'], metadata['salience']
metadata.columns = ['file', 'fold', 'class_id', 'class']
metadata['augment'] = 'none'


In [12]:
# Concat the two dataframes
full_meta = pd.concat([metadata, new_meta])

# Verify lengths
if (len(full_meta) == len(metadata) + len(new_meta)):
    print("Dataframes merged correctly!")
else:
    print("Error! Lengths do not match.")

print("Initial data:", len(metadata))
print("New data:", len(new_meta))
print("Merged data:", len(full_meta))

Dataframes merged correctly!
Initial data: 8732
New data: 96052
Merged data: 104784


/home/eduugr/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


### Save the new dataset

In [16]:
# Save the new metadata
full_meta.to_csv(metadata_augmented_path, index=False, encoding="utf-8")

### External validation files compilation code

In [5]:
import os
import pandas as pd

base_path = os.path.abspath("./data/evaluation/")

def get_external_samples(path):
    files = []
    labels = []

    # Iterate category directories
    dirs = os.listdir(base_path)
    for d in dirs:
        category = d
        category_path = os.path.join(base_path, d)

        # Iterate files
        dir_files = os.listdir(category_path)

        for file in dir_files:
            file_path = os.path.join(category_path, file)
            files.append(file_path)
            labels.append(category)

    # Create dataframe
    return pd.DataFrame({ 'class': labels, 'file': files })


external = get_external_samples(base_path)




,class,file
46,jackhammer,/home/edu/Projects/nn-audio-classifier/data/ev...
47,jackhammer,/home/edu/Projects/nn-audio-classifier/data/ev...
48,jackhammer,/home/edu/Projects/nn-audio-classifier/data/ev...
49,jackhammer,/home/edu/Projects/nn-audio-classifier/data/ev...
50,jackhammer,/home/edu/Projects/nn-audio-classifier/data/ev...
